In [1]:
'''Linear algebra'''
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

'''Data visualisation'''
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot # cf. 5.2

'''Preprocessing'''
from sklearn.impute import SimpleImputer # cf. 5.2
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,LabelEncoder,OneHotEncoder # cf. 5.2

'''Deep learning'''
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

'''Pipeline'''
from sklearn.pipeline import Pipeline,FeatureUnion # cf. 5.6
from sklearn.compose import ColumnTransformer,make_column_selector # cf. 5.6
from sklearn.preprocessing import FunctionTransformer # cf. 5.6
from sklearn.base import TransformerMixin,BaseEstimator # cf. 5.6

# Data preprocessing

## Loading dataset

In [2]:
dataset = pd.read_csv('../raw_data/ytrain_raw.csv')

/Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dataset.head()

,timestamp,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,S62-T3,S14-T3,S8-T1,S8-T3,S34-T3,S6-T2,S29-T3,S64-T1,S11-T1,S76-T3,S59-T1,S58-T1,S58-T3,S21-T3,S96-T3,S30-T1,S77-T1,S35-T3,S71-T3,S89-T1,S51-T3,S51-T1,S38-T3,S79-T1,S79-T3,S33-T3,S33-T1,S50-T3,S1-T3,S49-T1,S60-T3,S48-T2,S98-T3,S3-T3,S3-T1,S70-T3,S17-T1,S78-T1,S80-T3,S66-T3,S27-T3,S27-T1,S92-T3,S12-T2,S7-T3,S19-T3,S23-T1,S86-T1,S67-T3,S25-T3,S10-T3,S10-T1,S20-T3,S31-T1,S41-T3,S29-T1,S32-T1,S26-T1,S53-T1,S4-T3,S82-T1,S82-T3,S91-T1,S91-T3,S30-T3,S24-T1,S99-T1,S99-T3,S88-T1,S74-T3,S39-T3,S45-T2,S38-T1,S42-T2,S42-T1,S1-T2,S84-T1,S46-T1,S68-T1,S68-T3,S60-T1,S48-T3,S13-T3,S80-T2,S66-T1,S92-T1,S5-T3,S12-T3,S12-T1,S69-T3,S2-T3,S22-T1,S65-T1,S67-T1,S6-T3,S31-T3,S9-T1,S64-T2,S76-T1,S26-T3,S72-T1,S37-T1,S37-T3,S15-T1,S82-T2,S35-T1,S88-T2,S18-T3,S18-T1,S18-T2,S39-T1,S45-T3,S95-T1,S49-T3,S49-T2,S46-T3,S98-T1,S13-T1,S83-T1,S66-T2,S63-T3,S22-T3,S22-T2,S47-T1,S94-T1,S64-T3,S11-T3,S76-T2,S59-T3,S57-T1,S24-T3,S45-T1,S38-T2,S50-T1,S84-T3,S3-T2,S70-T1,S97-T1,S93-T3,S63-T1,S69-T1,S69-T2,S23-T3,S52-T2,S52-T1,S86-T2,S34-T1,S87-T3,S75-T1,S75-T3,S53-T2,S53-T3,S4-T1,S59-T2,S72-T3,S58-T2,S15-T2,S89-T2,S89-T3,S95-T3,S1-T1,S81-T1,S93-T2,S56-T1,S47-T3,S67-T2,S28-T3,S34-T2,S6-T1,S9-T3,S87-T1,S32-T3,S26-T2,S15-T3,S57-T3,S77-T3,S36-T3,S42-T3,S46-T2,S97-T3,S17-T3,S93-T1,S83-T3,S85-T2,S62-T1,S20-T1,S41-T1,S74-T1,S70-T2,S5-T1,S52-T3,S65-T3,S96-T1,S71-T1,S84-T2,S17-T2,S86-T3,S14-T2,S14-T1,S8-T2,S29-T2,S72-T2,S91-T2,S74-T2,S33-T2,S81-T3,S48-T1,S78-T3,S80-T1,S85-T1,S57-T2,S77-T2,S88-T3,S36-T1,S50-T2,S81-T2,S78-T2,S63-T2,S87-T2,S30-T2,S24-T2,S83-T2,S10-T2,S4-T2,S71-T2,S36-T2,S79-T2,S16-T2,S94-T2,S9-T2,S21-T1,S92-T2,S11-T2,S51-T2,S60-T2,S19-T2,S62-T2,S31-T2,S96-T2,S27-T2,S23-T2,S28-T2,S37-T2,S39-T2,S68-T2,S56-T2,S41-T2,S35-T2,S7-T2,S75-T2,S20-T2,S95-T2,S5-T2,S2-T2,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
0,2019-11-25T00:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Available,Passive,Available,Available,Charging,Available,Available,Available,Passive,Available,Available,Available,Available,Offline,Passive,Available,Available,Passive,Available,Available,Available,Available,Passive,Available,Available,Charging,Available,Charging,Down,Available,Offline,Offline,Offline,Offline,Offline,Offline,Offline,Offline,Offline,Available,Offline,Offline,Down,Available,Available,Available,Available,Available,Available,Available,Down,Down,Available,Available,Available,Available,Available,Available,Charging,Available,Available,Offline,Offline,Offline,Available,Available,Available,Available,Available,Passive,Charging,Charging,Charging,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Available,Offline,Available,Available,Available,Available,Available,Passive,Charging,Available,Passive,Down,Available,Down,Available,Offline,Offline,Offline,Passive,Charging,Charging,Charging,Charging,Down,Down,Down,Offline,Available,Offline,Available,Passive,Charging,Available,Available,Available,Available,Available,Passive,Available,Available,Available,Available,Available,Available,Offline,Offline,Offline,Offline,Offline,Available,Offline,Offline,Down,Down,Passive,Passive,Available,Available,Available,Charging,Available,Available,Available,Available,Available,Down,Down,Offline,Available,Offline,Offline,Offline,Available,Offline,Offline,Available,Available,Available,Available,Available,Offline,Available,Available,Available,Available,Down,Down,Offline,Offline,Available,Passive,Charging,Offline,Offline,Available,Available,Available,Available,Offline,Down,Down,Offline,Available,Charging,Available,Passive,Available,Charging,Available,Charging,Passive,Down,Charging,Passive,Available,Available,Passive,Charging,Charging,Charging,Available,Charging,Available,Available,Offline,Available,Charging,Charging,Available,Offline,Offline,Offlin

In [4]:
dataset.nunique().sort_values(ascending=False)

timestamp    31232
S15-T3           5
S45-T1           5
S50-T1           5
S3-T2            5
             ...  
S47-T1           1
S22-T2           1
S22-T3           1
S19-T2           1
S60-T1           1
Length: 274, dtype: int64

## Reduce size of dataset

In [5]:
cols_test = list(dataset.nunique().sort_values(ascending=False)[0:12].index)
cols_test

['timestamp',
 'S15-T3',
 'S45-T1',
 'S50-T1',
 'S3-T2',
 'S97-T1',
 'S93-T3',
 'S75-T1',
 'S75-T3',
 'S53-T2',
 'S53-T3',
 'S59-T2']

In [6]:
df_test = dataset[cols_test]
df_test

,timestamp,S15-T3,S45-T1,S50-T1,S3-T2,S97-T1,S93-T3,S75-T1,S75-T3,S53-T2,S53-T3,S59-T2
0,2019-11-25T00:00:00+00:00,Available,Available,Available,Passive,Available,Available,Offline,Offline,Down,Down,Passive
1,2019-11-25T00:15:00+00:00,Available,Available,Available,Passive,Available,Available,Offline,Offline,Down,Down,Passive
2,2019-11-25T00:30:00+00:00,Available,Available,Available,Passive,Available,Available,Offline,Offline,Down,Down,Passive
3,2019-11-25T00:45:00+00:00,Available,Available,Available,Passive,Available,Available,Offline,Offline,Offline,Offline,Passive
4,2019-11-25T01:00:00+00:00,Available,Available,Available,Passive,Available,Available,Offline,Offline,Down,Down,Passive
...,...,...,...,...,...,...,...,...,...,...,...,...
31227,2020-11-08T22:45:00+00:00,Available,Charging,Down,Passive,Available,Available,Available,Available,Charging,Available,NaN
31228,2020-11-08T23:00:00+00:00,Available,Available,Down,Passive,Available,Available,Available,Available,Charging,Available,NaN
31229,2020-11-08T23:15:00+00:00,Available,Available,Down,Passive,Available,Available,Available,Available,Charging,Available,NaN
31230,2020-11-08T23:30:00+00:00,Available,Available,Down,Passive,Available,Available,Available,Available,Charging,Available,NaN


## Preprocessing

In [15]:
df_test = df_test.fillna(value='Down')
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

## Encoding X

In [8]:
station_ids = list(set(df_test.columns) - {'timestamp'})
station_ids

['S97-T1',
 'S3-T2',
 'S15-T3',
 'S75-T1',
 'S75-T3',
 'S50-T1',
 'S53-T2',
 'S45-T1',
 'S53-T3',
 'S59-T2',
 'S93-T3']

In [9]:
for station_id in station_ids:
    print(df_test[station_id].unique())

['Available' 'Passive' 'Charging' 'Offline' 'Down']
['Passive' 'Available' 'Offline' 'Down' 'Charging']
['Available' 'Passive' 'Charging' 'Offline' 'Down']
['Offline' 'Passive' 'Charging' 'Available' 'Down']
['Offline' 'Available' 'Charging' 'Passive' 'Down']
['Available' 'Passive' 'Charging' 'Offline' 'Down']
['Down' 'Offline' 'Passive' 'Available' 'Charging']
['Available' 'Charging' 'Passive' 'Offline' 'Down']
['Down' 'Offline' 'Passive' 'Available' 'Charging']
['Passive' 'Available' 'Offline' 'Charging' 'Down']
['Available' 'Charging' 'Offline' 'Passive' 'Down']


In [10]:
X = []
y = []
test_period = 14*24
for stadion_id in station_ids:
    df_station_ohe = pd.get_dummies(df_test[stadion_id],dtype='int32')
    X.append(np.array(df_station_ohe[:-test_period]))
    y.append(np.array(df_station_ohe[-test_period:]))
X = np.array(X)
y = np.array(y)

In [19]:
pd.DataFrame(y[0])

,0,1,2,3,4
0,0,1,0,0,0
1,0,0,0,0,1
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
331,1,0,0,0,0
332,1,0,0,0,0
333,1,0,0,0,0
334,1,0,0,0,0


In [11]:
print(X.shape)
print(y.shape)

(11, 30896, 5)
(11, 336, 5)


In [12]:
for i in range(len(X)):
    print(np.unique(X[0]))

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


## Model

In [13]:
# ------ Model
model = Sequential()

model.add(SimpleRNN(20, activation='tanh'))
model.add(Dense(10,activation='relu'))
model.add(Dense(4, activation='softmax'))

# ----- Compilation
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics='accuracy')

In [14]:
model.fit(X, y,
          epochs=100,
          batch_size=32,
          verbose=1)

Epoch 1/100


TypeError: in user code:

    /Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.